# Decorators and Closures

## Decorators 101

In [ ]:
# A decorator is a callable that takes another function as an argument

# assuming an existing decorator named decorate
@decorate
def target():
    print('running target()')

# has the same effect as writing this
def target():
    print('running target()')

target = decorate(target)

In [3]:
# A decorator usually replaces a function with a different one
def deco(func):
    def inner():
        print('running inner')
    return inner  #1

@deco
def target():  #2
    print('running target()')

target()  #3

running inner


In [4]:
target  #4

<function __main__.deco.<locals>.inner()>

## When Python Executes Decorators

see registration.py module

## Variable Scope Rules

In [7]:
b = 6

def f1(a):
    print(a)
    print(b)

f1(3)

3
6


In [13]:
b = 6

def f2(a):
    global b
    print(a)
    print(b)
    b = 9

f2(3)
b

3
6


9

In [15]:
from dis import dis

dis(f1)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [16]:
dis(f2)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## Closures

In [18]:
# average_oo.py: a class to calculate a running average
from typing import Any

class Averager:
    def __init__(self) -> None:
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))


10.0
10.5
11.0


In [19]:
# average.py: a high-order function to calculate a running average
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [20]:
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
avg.__closure__

(<cell at 0x112795900: list object at 0x118cd9e00>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## The nonlocal Declaration

In [25]:
# A broken higher-order function to calculate a running average without keeping all history
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0

## Implementing a Simple Decorator

In [3]:
import time
from clockdeco0 import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.132)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.132)
[0.12787219s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000056s] factorial(1) -> 1
[0.00001371s] factorial(2) -> 2
[0.00002147s] factorial(3) -> 6
[0.00002832s] factorial(4) -> 24
[0.00003631s] factorial(5) -> 120
[0.00004428s] factorial(6) -> 720
6! = 720


## Decorators in the Standard Library

### Memoization with functools.cache

In [ ]:
import functools

from clockdeco0 import clock


@functools.cache  #1
@clock  #2
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(30))

### Using lru_cache

In [9]:
# The main advantage of @lru_cache is that its memory usage is bounded by the 
# maxsize parameter, which has a rather conservative default value of 128 - 
# which means the cache will hold at most 128 entries at any time.
from functools import lru_cache

@lru_cache
def costly_function(a, b):
    ...

# or

@lru_cache()
def costly_function(a, b):
    ...

# in both cases, the default parameters wold be used

## maxsize=128
# Sets the maximum number of entries to stored. After the cache is full, the last
# recently used entry is discarded to make room for each new entry. For optimal
# performance, maxsize should be a power of 2. If you pass maxsize=None, the LRU
# is disabled, so the cache works faster but entries are never discarded, which
# may consumes too much memory. That's what @functools.cache does.

## typed=False
# Determines whether the results of different arguments types are stored separately.
# For example, in the default setting, float and integer arguments that are considered
# equal are stored only once, so there would be a single entry for the calls f(1) and
# f(1.0). If typed=True, those arguments would produce different entries, possibly
# storing distinct results.

@lru_cache(maxsize=2**20, typed=True)
def costly_function(a, b):
    ...


### Single Dispatch Generic Functions

#### Function singledispatch

In [7]:
# @singledispatch creates a custom @htmlize.register to bundle several functions into a generic function
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch  #1
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f"<pre>{content}</pre>"

@htmlize.register  #2
def _(text: str) -> str:  #3
    content = html.escape(text).replace('\n', '<br/>\n')
    return f'<p>{content}</p>'

@htmlize.register  #4
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register  #5
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register  #6
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)  #7
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)  #8
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})<pre>'

In [4]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [5]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [8]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br/>\n- a game</p>'

In [9]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [10]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [11]:
htmlize(True)

'<pre>True</pre>'

In [12]:
htmlize(fractions.Fraction(2, 3))

'<pre>2/3</pre>'

In [13]:
htmlize(2/3)

'<pre>0.6666666666666666 (2/3)<pre>'

In [14]:
htmlize(decimal.Decimal('0.02380952'))

'<pre>0.02380952 (1/42)<pre>'

## Parameterized Decorators

### A Parameterized Registration Decorator

In [15]:
import registration_param

running register(active=False)->decorate(<function f1 at 0x1099541f0>)
running register(active=True)->decorate(<function f2 at 0x1099548b0>)


In [16]:
registration_param.registry

{<function registration_param.f2()>}

In [17]:
from registration_param import *

In [18]:
registry

{<function registration_param.f2()>}

In [19]:
register()(f3)

running register(active=True)->decorate(<function f3 at 0x103686170>)


<function registration_param.f3()>

In [20]:
registry

{<function registration_param.f2()>, <function registration_param.f3()>}

In [21]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x1099548b0>)


<function registration_param.f2()>

In [22]:
registry

{<function registration_param.f3()>}

### The Parameterized Clock Decorator

In [23]:
# clockdemo_param_demo1.py
import time
from clockdeco_param import clock

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12785359399276786s
snooze: 0.12747396199847572s
snooze: 0.12724219799565617s


In [9]:
# clockdemo_param_demo2.py
import time
from clockdeco_param import clock

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s


### A Class-Based Clock Decorator

In [12]:
import time
from clockdeco_cls import clock

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

TypeError: 'NoneType' object cannot be interpreted as an integer